In [1]:
import pandas as pd
import sqlalchemy
import mysql.connector

#Pour que python reconnaisse un dossier comme un module il faut ajouter en son sein un fichier __init__.py 
#et également l'ajouter au PATH

import sys  

sys.path.insert(0, "/home/apprenant/Desktop/AmericanDream/")


#On peut ensuite importer le dossier comme un module

from src.d00_utils.mysql_utils import mysql_connect, save_to_mysql
connect = mysql_connect()

Importation des données depuis my sql


##1. Sélection des colonnes
J'affiche un échantillon de mes données pour analyser la pertinence des différentes colonnes    

In [2]:
data = pd.read_sql("Analyst",con=connect)
data2 = pd.read_sql("Salary",con=connect)

In [4]:
print(data2.head())
print(data2.columns)

   index  Survey Year           Timestamp  SalaryUSD        Country  \
0      0         2020 2020-01-04 18:50:34   115000.0  United States   
1      1         2020 2020-01-04 10:43:02   100000.0  United States   
2      2         2020 2020-01-04 09:51:46   100000.0          Spain   
3      3         2020 2020-01-04 01:08:54    70000.0  United States   
4      4         2020 2020-01-03 15:28:54   110000.0  United States   

  PostalCode       PrimaryDatabase  YearsWithThisDatabase  \
0         03  Microsoft SQL Server                     15   
1       None                 Other                      6   
2      28046  Microsoft SQL Server                      2   
3      94133  Microsoft SQL Server                      3   
4      95354                Oracle                     30   

                                      OtherDatabases    EmploymentStatus  ...  \
0  Microsoft SQL Server, MongoDB, Azure SQL DB (a...  Full time employee  ...   
1                                      MySQL

In [5]:
data2 = data2[['index','SalaryUSD','Country','PostalCode','EmploymentStatus','JobTitle','ManageStaff','YearsWithThisTypeOfJob',
          'HowManyCompanies','OtherPeopleOnYourTeam', 'CompanyEmployeesOverall',
          'PopulationOfLargestCityWithin20Miles','EmploymentSector', 'LookingForAnotherJob', 'CareerPlansThisYear',
          'Gender', 'OtherJobDuties']]

##2. Valeurs manquantes
Recherche des valeurs manquantes

In [6]:
print(data2.isnull().sum()) 
print(data2.shape)

index                                      0
SalaryUSD                                  0
Country                                    0
PostalCode                              1390
EmploymentStatus                           0
JobTitle                                   0
ManageStaff                                0
YearsWithThisTypeOfJob                     0
HowManyCompanies                           0
OtherPeopleOnYourTeam                      0
CompanyEmployeesOverall                    0
PopulationOfLargestCityWithin20Miles       0
EmploymentSector                           0
LookingForAnotherJob                       0
CareerPlansThisYear                        0
Gender                                     0
OtherJobDuties                           620
dtype: int64
(8627, 17)



Il me manque 1390 valeurs pour PostalCode et 620 pour OtherJobDUties sur un total de 8627 lignes. Ces deux colonnes ne sont pas au centre de mon études donc ça se vaut pas le cout de supprimer toute la ligne pour elles

traitement de Postal code
Cela ne ferait pas de sens de remplacer le code postal par un code postal arbitraire

Traitement de OtherJobDuties

In [7]:
data2 = data2[data2.Country == 'United States']

In [8]:
print(data2)

      index  SalaryUSD        Country PostalCode  \
0         0   115000.0  United States         03   
1         1   100000.0  United States       None   
3         3    70000.0  United States      94133   
4         4   110000.0  United States      95354   
5         5    79500.0  United States      78727   
...     ...        ...            ...        ...   
8609   8609    66000.0  United States  Not Asked   
8611   8611    96000.0  United States  Not Asked   
8612   8612    85000.0  United States  Not Asked   
8614   8614   137500.0  United States  Not Asked   
8626   8626   200000.0  United States  Not Asked   

                                       EmploymentStatus  \
0                                    Full time employee   
1                                    Full time employee   
3                                    Full time employee   
4                                    Full time employee   
5                                    Full time employee   
...                  

In [9]:
print(data2.dtypes)

index                                     int64
SalaryUSD                               float64
Country                                  object
PostalCode                               object
EmploymentStatus                         object
JobTitle                                 object
ManageStaff                              object
YearsWithThisTypeOfJob                    int64
HowManyCompanies                         object
OtherPeopleOnYourTeam                    object
CompanyEmployeesOverall                  object
PopulationOfLargestCityWithin20Miles     object
EmploymentSector                         object
LookingForAnotherJob                     object
CareerPlansThisYear                      object
Gender                                   object
OtherJobDuties                           object
dtype: object


Aucune de mes colonnes ne correspond à une date, je n'ai donc pas à faire de traitement

##3. Traitement des doublons

In [10]:

print(data2.duplicated().value_counts())

False    5680
dtype: int64


Aucune des colonnes de ma base n'est dupliquée, je vérifie cependant que c'est également le cas sur la colonne index

In [11]:

print(data2.duplicated('index').value_counts())

False    5680
dtype: int64


On est bon.

to be continued
Avant de partir je sauvegarde mon travail dans my sql

In [12]:
save_to_mysql(db_connect=connect,df_to_save=data2,df_name='Salary_01_v1.0')

#J'éffectue la même chose pour la 2eme tables

In [ ]:
print(data.head())
print(data.columns)

In [72]:
data = data[['index', 'Job Title', 'Salary Estimate',
       'Job Description', 'Company Name', 'Location','Headquarters',
       'Size', 'Type of ownership', 'Industry', 'Sector', 'Revenue']]

les colonnes que je vais garder pour mon étude.

In [73]:
print(data.isnull().sum()) 
print(data.shape)

index                0
Job Title            0
Salary Estimate      0
Job Description      0
Company Name         1
Location             0
Headquarters         0
Size                 0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
dtype: int64
(2253, 12)


La data est plutot clean une seul erreur je ne toucherai donc pas 

In [74]:
print(data.dtypes)


index                 int64
Job Title            object
Salary Estimate      object
Job Description      object
Company Name         object
Location             object
Headquarters         object
Size                 object
Type of ownership    object
Industry             object
Sector               object
Revenue              object
dtype: object


In [75]:
print(data.duplicated().value_counts())

False    2253
dtype: int64


In [13]:
print(data.duplicated('index').value_counts())

False    2253
dtype: int64


In [ ]:
save_to_mysql(db_connect=connect,df_to_save=data,df_name='Analyst_01_v1.0')

Je sauvegarde mon travail